In [1]:
# Importer les bibliothèques nécessaires

import pandas as pd
import numpy as np

# from sklearn.datasets import fetch_california_housing

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, RegressionPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, RegressionTestPreset
from evidently.tests import *

In [2]:
# Charger les données d'entraînement et de test à partir des fichiers CSV
data_train = pd.read_csv('./Data/app_train.csv')
data_test = pd.read_csv('./Data/app_test.csv')

In [3]:
# # Charger les données
# data_train = pd.read_csv("app_train.csv")
# data_test = pd.read_csv("app_test.csv")

# Supprimez ces lignes si elles ne sont pas nécessaires
# Elles désactivent simplement les avertissements de pandas
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Utilisez votre DataFrame client (client_data) à la place de la charge de données California Housing
client_data = data_train.copy()  # Assurez-vous d'avoir une copie de vos données

# Calculer l'âge à partir de la colonne "DAYS_BIRTH" dans le DataFrame client
client_data['Age'] = -client_data['DAYS_BIRTH'] // 365
# Calculer l'âge à partir de la colonne "DAYS_BIRTH" dans le DataFrame client
client_data['Age'] = -client_data['DAYS_BIRTH'] // 365

# Sélectionner les colonnes spécifiques
# selected_columns = ['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT']
selected_columns = ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT','Age']
# Créer un DataFrame avec les colonnes spécifiques
df_cleaned = client_data[selected_columns]

# Sélectionner uniquement les colonnes numériques
numeric_columns = df_cleaned.select_dtypes(include=['number'])

# Créer un nouveau DataFrame avec les colonnes numériques
client_data = df_cleaned[numeric_columns.columns]

# Assurez-vous que les colonnes correspondent à celles de votre DataFrame client
client_data.rename(columns={'TARGET': 'target'}, inplace=True)

# Remplacez les valeurs NaN par des zéros dans la colonne 'target' du dataframe de référence
client_data['target'].fillna(0, inplace=True)

# Ajoutez une colonne de prédiction basée sur la colonne 'target' avec un bruit aléatoire
client_data['prediction'] = client_data['target'] + np.random.normal(0, 5, client_data.shape[0])

client_data['prediction'].fillna(0, inplace=True)

# Créez un DataFrame de référence et actuel à partir de votre DataFrame client
reference = client_data.sample(n=5000, replace=False)
current = client_data.sample(n=5000, replace=False)

# Affichez les DataFrames de référence et actuels
print("DataFrame de Référence (Reference):")
print(reference)
print("\nDataFrame Actuel (Current):")
print(current)
reference = reference.astype(float)
current = current.astype(float)
# Supprimer les lignes contenant des valeurs nulles
reference = reference.dropna()
current = current.dropna()


DataFrame de Référence (Reference):
      SK_ID_CURR  target  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  Age  \
3871      104530       0             0          112500.0   1528200.0   52   
806       100923       0             2          270000.0    284256.0   33   
4279      105004       0             1           90000.0    242595.0   39   
8013      109343       0             0          202500.0    834048.0   56   
5518      106464       0             0          108000.0    180000.0   21   
...          ...     ...           ...               ...         ...  ...   
174       100203       0             0           54000.0    119925.0   68   
5984      107007       0             0           67500.0    545040.0   23   
998       101155       0             0          315000.0   1288350.0   44   
7567      108824       0             2          171000.0   1078200.0   33   
3080      103597       0             0          306000.0    605439.0   62   

      prediction  
3871    0.879257  
8

In [4]:
# Définition du rapport initial avec des métriques de référence
report = Report(metrics=[
    DataDriftPreset(),  # Utilise la métrique de dérive de données prédéfinie
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec des métriques spécifiques sur la colonne 'AveRooms'
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),  # Résume les statistiques de la colonne
    ColumnQuantileMetric(column_name='AMT_INCOME_TOTAL', quantile=0.25),  # Calcule le quantile de la colonne
    ColumnDriftMetric(column_name='AMT_INCOME_TOTAL'),  # Calcule la dérive de la colonne
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec des métriques générées pour plusieurs colonnes
report = Report(metrics=[
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile': 0.25}, columns=['AMT_INCOME_TOTAL', 'AMT_CREDIT']),
    # Génère des métriques de quantile pour les colonnes spécifiées
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec une variété de métriques pour différentes colonnes et une métrique prédéfinie
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),  # Résume les statistiques de la colonne
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile': 0.25}, columns='num'),  # Génère des métriques de quantile pour 'num'
    DataDriftPreset(),  # Utilise la métrique de dérive de données prédéfinie
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Conversion du rapport en dictionnaire
report.as_dict()

# Conversion du rapport en format JSON
report.json()

# Explication du code :
# Ce code Python utilise la bibliothèque "datadrift" pour créer des rapports de dérive de données entre un jeu de données de référence (reference) et un jeu de données actuel (current).
# Le code commence par importer les bibliothèques nécessaires.
# Il définit ensuite plusieurs rapports avec différentes métriques pour évaluer la dérive de données, telles que la dérive globale des données, les statistiques de colonnes, les quantiles, etc.
# Chaque rapport est exécuté avec les données de référence et les données actuelles, puis le rapport est affiché.
# À la fin, le code montre comment convertir le rapport en dictionnaire et en format JSON.
# Ces rapports sont utiles pour surveiller et analyser la stabilité des données au fil du temps, ce qui est essentiel dans de nombreuses applications, notamment pour détecter des anomalies ou des changements dans les données.


'{"version": "0.4.9", "metrics": [{"metric": "ColumnSummaryMetric", "result": {"column_name": "AMT_INCOME_TOTAL", "column_type": "num", "reference_characteristics": {"number_of_rows": 5000, "count": 5000, "missing": 0, "missing_percentage": 0.0, "mean": 166538.61, "std": 89794.36, "min": 27000.0, "p25": 112500.0, "p50": 144000.0, "p75": 202500.0, "max": 1350000.0, "unique": 195, "unique_percentage": 3.9, "infinite_count": 0, "infinite_percentage": 0.0, "most_common": 135000.0, "most_common_percentage": 11.52}, "current_characteristics": {"number_of_rows": 5000, "count": 5000, "missing": 0, "missing_percentage": 0.0, "mean": 167225.35, "std": 94568.98, "min": 27000.0, "p25": 112500.0, "p50": 144000.0, "p75": 202500.0, "max": 1350000.0, "unique": 222, "unique_percentage": 4.44, "infinite_count": 0, "infinite_percentage": 0.0, "most_common": 135000.0, "most_common_percentage": 11.64}}}, {"metric": "ColumnQuantileMetric", "result": {"column_name": "CNT_CHILDREN", "column_type": "num", "qua

In [5]:
# Suite de Tests

# Création d'une suite de tests comprenant plusieurs tests spécifiques
tests = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(), # Test du nombre de colonnes avec des valeurs manquantes
    TestNumberOfRowsWithMissingValues(),  # Test du nombre de lignes avec des valeurs manquantes
    TestNumberOfConstantColumns(),  # Test du nombre de colonnes constantes
    TestNumberOfDuplicatedRows(),  # Test du nombre de lignes dupliquées
    TestNumberOfDuplicatedColumns(),  # Test du nombre de colonnes dupliquées
    TestColumnsType(),   # Test du type de données des colonnes
    TestNumberOfDriftedColumns(),  # Test du nombre de colonnes présentant une dérive de données
])

# Exécution des tests avec les données de référence (reference) et les données actuelles (current)
tests.run(reference_data=reference, current_data=current)

# Affichage des résultats des tests
tests
# Création d'une nouvelle suite de tests spécifiques à la performance sans cible
suite = TestSuite(tests=[
    NoTargetPerformanceTestPreset(), # Test de performance sans cible
])
# Exécution des tests avec les données de référence (reference) et les données actuelles (current)
suite.run(reference_data=reference, current_data=current)
# Affichage des résultats des tests
suite

In [6]:
# Conversion de la colonne 'NAME_CONTRACT_TYPE' en type de données chaîne de caractères (str) dans le DataFrame de référence

# reference['NAME_CONTRACT_TYPE'] = reference['NAME_CONTRACT_TYPE'].astype(str)
# Créez un ensemble de tests en utilisant les objets de colonne du DataFrame
suite = TestSuite(tests=[
#     TestColumnDrift(df_reference['NAME_CONTRACT_TYPE']),
    
# Création d'une suite de tests en utilisant les objets de colonne du DataFrame

# Remarque : Le test TestColumnDrift a été commenté car la colonne 'NAME_CONTRACT_TYPE' a été convertie en type de données chaîne de caractères.
# TestColumnDrift(df_reference['NAME_CONTRACT_TYPE']),

# Test de dérive de colonne pour la colonne 'target'
    TestColumnDrift(column_name='target'),

# Test de moyenne dans N sigmas pour la colonne 'Age'
    TestMeanInNSigmas('Age'),

# Test de performance sans cible pour les colonnes spécifiques 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'CNT_CHILDREN'
    NoTargetPerformanceTestPreset(columns=['AMT_INCOME_TOTAL','AMT_CREDIT', 'CNT_CHILDREN'])
])


In [7]:
# Exécution des tests avec les données de référence et les données actuelles
suite.run(reference_data=reference, current_data=current)
# Affichez les résultats de la suite de tests
suite

In [8]:
# Créez une suite de tests avec divers tests prédéfinis pour évaluer la qualité et la stabilité des données
suite = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(), # Teste le nombre de colonnes avec des valeurs manquantes
    TestNumberOfRowsWithMissingValues(),  # Teste le nombre de lignes avec des valeurs manquantes
    TestNumberOfConstantColumns(),  # Teste le nombre de colonnes constantes
    TestNumberOfDuplicatedRows(), # Teste le nombre de lignes dupliquées
    TestNumberOfDuplicatedColumns(), # Teste le nombre de colonnes dupliquées
    TestColumnsType(),  # Teste le type des colonnes
    TestNumberOfDriftedColumns(),  # Teste le nombre de colonnes en dérive
    TestColumnDrift('AMT_CREDIT'), # Teste la dérive de la colonne 'AMT_CREDIT'
    TestShareOfOutRangeValues('AMT_CREDIT'),  # Teste la part des valeurs hors de portée dans 'AMT_CREDIT'
    DataStabilityTestPreset(),  # Test de stabilité des données
    RegressionTestPreset()     # Test de régression
])

In [9]:
# Exécutez la suite de tests avec les données de référence et les données actuelles
suite.run(reference_data=reference, current_data=current)
# Affichez les résultats de la suite de tests
suite

In [10]:
# Convertissez la suite de tests en dictionnaire
suite.as_dict()

# Convertissez la suite de tests en format JSON
suite.json()

'{"version": "0.4.9", "tests": [{"name": "The Number of Columns With Missing Values", "description": "The number of columns with missing values is 0. The test threshold is lte=0.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": 0}, "value": 0.0}}, {"name": "The Number Of Rows With Missing Values", "description": "The number of rows with missing values is 0. The test threshold is lte=0 \\u00b1 1e-12.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": {"value": 0.0, "relative": 0.1, "absolute": 1e-12}}, "value": 0.0}}, {"name": "Number of Constant Columns", "description": "The number of constant columns is 0. The test threshold is lte=0.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": 0}, "value": 0.0}}, {"name": "Number of Duplicate Rows", "description": "The number of duplicate rows is 0. The test threshold is eq=0 \\u00b1 1e-12.", "status": "SUCCESS", "group": "data_integrity"

In [11]:
# Enregistrez la suite de tests au format HTML
suite.save_html('test_suite_correct.html')

In [12]:
# Transformer toutes les colonnes en float
reference = reference.astype(float)

In [13]:
# ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT','Age']
# Importer la bibliothèque Pandas si ce n'est pas déjà fait


# Remplacer 'df' par le nom de votre DataFrame si ce n'est pas déjà fait
valeurs_uniques_r = reference['SK_ID_CURR'].unique()

# Afficher les valeurs uniques de la colonne 'target'
print(valeurs_uniques_r)
# Remplacer 'df' par le nom de votre DataFrame si ce n'est pas déjà fait
valeurs_uniques_c = current['SK_ID_CURR'].unique()

# Afficher les valeurs uniques de la colonne 'target'
print(valeurs_uniques_c)

[104530. 100923. 105004. ... 101155. 108824. 103597.]
[105773. 103384. 108563. ... 110956. 104313. 111022.]


In [14]:
# Votre DataFrame
data = reference.copy()

# Vérification de la présence de NaN dans le DataFrame
has_nan = data.isnull().any().any()

# Vérification de la présence de valeurs non numériques dans le DataFrame
is_numeric = data.applymap(np.isreal).all().all()

if not has_nan and is_numeric:
    print("Toutes les colonnes contiennent des valeurs numériques et il n'y a pas de NaN.", has_nan,is_numeric)
else:
    print("Le DataFrame contient des valeurs non numériques ou des NaN.")


Toutes les colonnes contiennent des valeurs numériques et il n'y a pas de NaN. False True
